In [1]:
!pip install backtesting

In [82]:
from backtesting import Strategy,Backtest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pyfolio as pf
import csv; import datetime; import pytz
# from technical_indicators import DONCH
from historical_data import exchange_data,write_to_csv,to_unix_time
# import backtest
import warnings
warnings.filterwarnings('ignore')

# ==========Initial trade parameters =============
symbol = 'BTC/USD'
timeframe = '1h'
exchange='kraken'
trading_qty = 1.0
trn_cost = 0.0016       # === Transaction cost = 0.26%
slippage = 0.001        # === Slippage = 0.1%
borrow_cost = 0.0026    # === Assuming every short trade is 100
since = '2017-01-01 00:00:00'
hist_start_date = int(to_unix_time(since))
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']

#Donchian Channel
def DONCH(df, n):

    df['Upper_DC'] = df['High'].shift(1).rolling(window=n).max().fillna(0)
    df['Lower_DC'] = df['Low'].shift(1).rolling(window=n).min().fillna(0)
    df['Upper_DC']=df['Upper_DC'].shift(3)
    df['Lower_DC']=df['Lower_DC'].shift(3)

    return df['Upper_DC'], df['Lower_DC']


class System(Strategy):

    kraken = exchange_data(exchange=exchange,
                           symbol=symbol,
                           timeframe=timeframe, since=hist_start_date)
    write_to_csv(kraken, symbol, exchange)
    data = pd.DataFrame(kraken, columns=header)

#     data = pd.read_csv("kraken-BTCUSD-1h.csv",encoding='utf-8')

    period = 20
    
    def init(self):
        self.upper_donch, self.lower_donch = self.I(DONCH,self.data, self.period)
        print(self.upper_donch)
    def next(self):
        price=self.data.Close.iloc[-1]
        if (not self.position and
            price > self.upper_donch[-1]):
            self.buy(sl=price < self.lower_donch[-1])
            
        elif (not self.position and
             price < self.lower_donch[-1]):
            self.sell(sl=price > self.upper_donch[-1])
        
        
backtest = Backtest(data, System, cash=10000, commission=.002)
backtest.run()
backtest.plot()

[   nan    nan    nan    0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.  8220.  8220.  8220.  8220.  8220.  8220.  8220.
 8220.  8220.  8220.  8220.  8220.  8220.  8220.  8116.5 8116.5 8116.5
 8116.5 8116.5 8116.5 8116.5 8116.5 8111.5 8107.5 8107.5 8105.9 8080.9
 8075.6 7941.8 7923.9 7906.1 7890.8 7890.  7870.  7775.8 7719.3 7719.3
 7719.3 7719.3 7719.3 7719.3 7719.3 7719.3 7719.3 7719.3 7719.3 7719.3
 7719.3 7719.3 7719.3 7719.3 7719.3 7518.3 7402.2 7399.8 7399.8 7399.8
 7399.8 7399.8 7399.8 7399.8 7399.8 7399.8 7399.8 7356.5 7349.6 7339.9
 7339.9 7339.9 7339.9 7339.9 7339.9 7344.  7344.  7344.  7344.  7344.
 7344.  7344.  7344.  7344.  7344.  7344.  7344.  7344.  7344.  7344.
 7344.  7344.  7344.  7344.  7344.  7293.4 7293.4 7280.  7280.  7280.
 7280.  7280.  7280.  7280.  7280.  7260.  7212.9 7196.6 7178.  7123.7
 7123.7 7243.8 7285.  7285.  7296.8 7372.6 7372.6 7372.6 7372.6 7372.6
 7372.6 7372